Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] SubmitUse hyperparameters that match the classifier, such as randomforestclassifier__ ...
Use a metric for classification, such as scoring='accuracy' your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd
DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
# Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.

# You can't just copy from the lesson notebook to this assignment.
## - Because the lesson was regression, but the assignment is classification.
## - Because the lesson used TargetEncoder, which doesn't work as-is for multi-class classification.

# Use hyperparameters that match the classifier, such as randomforestclassifier__ ...
# Use a metric for classification, such as scoring='accuracy'

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [5]:
### this was the original Wrangle from previous assignment
## my Hypothises was that rows with missing data were low quality entries and should be discarded.
## also I droped features that held mostly the same values.
## in 222 I found that droping rows was a horrible idea
# in  223 I plan to custom label all rows that have missing values

# another theory I may want to test out is that its uneccesary to drop any columns as random forest may select the best features regardless

def Base_Wrangle(local_xandy):
    xandy = local_xandy.copy(deep = True)
    to_drop=['num_private','wpt_name','recorded_by','date_recorded','extraction_type_group','extraction_type_class','payment_type','quality_group','quantity_group','source_type','waterpoint_type_group']
    
    for feature in to_drop:        
        xandy.drop(columns=[feature],inplace=True)
    
    
    print('start: '+str(xandy.shape))
    
    # transform amount_tsh    
    xandy.amount_tsh= xandy.amount_tsh.apply(lambda x: min(x,200000))    
    xandy.amount_tsh+=1
    xandy.amount_tsh=np.log(xandy.amount_tsh)
   
    #make features categorical
    #make region_code categorical
    xandy.region_code = xandy.region_code.astype(str)
    
    #make district_code cateogorical
    xandy.district_code = xandy.district_code.astype(str)

    #replace missing scheme_name    
    xandy.loc[xandy.scheme_name.isna(),'scheme_name'] ="missing"
    print(xandy.shape)

    #relace missing in funder
    xandy.loc[xandy.funder.isna(),'funder'] ="missing"
    #xandy = xandy[xandy.funder.isna()==False]
    

    # continuous variable with actual missing population data
    xandy.loc[xandy.population == 0,'population'] = xandy[xandy.population != 0].population.mean()

    xandy.loc[xandy.public_meeting.isna(),'public_meeting'] ="missing"    

    xandy.loc[xandy.permit.isna(),'permit'] ="missing"    

    #continuous variable with some categorical information...possibly
    xandy['construction_year_missing'] = xandy.construction_year==0
    #put the 0's at mean just in case
    mean = xandy.loc[xandy.construction_year != 0].construction_year.mean()
    xandy.loc[xandy.construction_year == 0,'construction_year'] = mean
    xandy.loc[xandy.construction_year.isna(),'construction_year']= mean
    
    xandy.loc[xandy.scheme_management.isna(),'scheme_management'] ="missing"        

    #TODO - verify these are not location at sea level
    xandy.loc[xandy.gps_height == 0,'gps_height']= train.loc[train.gps_height != 0].gps_height.mean()     
    xandy.loc[xandy.gps_height.isna(),'gps_height'] = train.loc[train.gps_height != 0].gps_height.mean()    
    
    xandy.loc[xandy.installer.isna(),'installer'] ="missing"    

    #mean the null island
    fltr = (xandy.longitude==0 )&(xandy.latitude==0 )
    
    xandy.loc[fltr,'longitude']= xandy.loc[fltr==False].longitude.mean()
    xandy.loc[fltr,'latitude'] = xandy.loc[fltr==False].latitude.mean()
    
    xandy.loc[xandy.subvillage.isna(),'subvillage'] ="missing"    
    
    print(xandy.shape)
    print("done")
    return xandy
          

In [6]:
wrangled_data = Base_Wrangle(train).copy(deep=True)
print(wrangled_data.isna().any())
print('-wrangled-')
print(wrangled_data.loc[wrangled_data.gps_height.isna(),'gps_height'] )
print('-train-')
print(train.loc[train.gps_height.isna(),'gps_height'] )

start: (59400, 30)
(59400, 30)
(59400, 31)
done
id                           False
amount_tsh                   False
funder                       False
gps_height                   False
installer                    False
longitude                    False
latitude                     False
basin                        False
subvillage                   False
region                       False
region_code                  False
district_code                False
lga                          False
ward                         False
population                   False
public_meeting               False
scheme_management            False
scheme_name                  False
permit                       False
construction_year            False
extraction_type              False
management                   False
management_group             False
payment                      False
water_quality                False
quantity                     False
source                       False
source_

In [7]:
def split_Xy(local_xandy,target = 'status_group'):
    xandy = local_xandy.copy(deep = True)    
    y=xandy[target]
    X=xandy.drop(columns=[target]).copy(deep=True)
    y=pd.DataFrame(y)
    return (X,y)
    
def split_data(local_xandy,target = 'status_group',_train_size=.8):
    split_Xy(local_xandy)
    what = train_test_split(
    X,y, train_size=_train_size,
    random_state=42,
    stratify=y[target])

    X_train, X_val, y_train, y_val=what
    y_train=y_train.to_numpy().reshape(-1,)
    y_val=y_val.to_numpy().reshape(-1,)
    return (X_train, X_val, y_train, y_val)
    

In [8]:
X_train,y_train = split_Xy(wrangled_data)

In [17]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

prepipe = make_pipeline(
    ce.OrdinalEncoder(),         
    StandardScaler(with_mean=True), 
)
prepipe.fit(X_train,y_train)
preped_data_train = prepipe.transform(X_train)



model = make_pipeline(
        #StandardScaler(), 
        RandomForestClassifier(),
        memory ='.\\memory\\',
        )

param_distributions = {
    #'standardscaler__with_mean': [True, False],     
    'randomforestclassifier__min_samples_leaf': range(1,3), 
    'randomforestclassifier__min_samples_split': range(2,4),     
    'randomforestclassifier__n_estimators':[1000],
    'randomforestclassifier__n_jobs':[1],
    'randomforestclassifier__max_depth':range(int(59*.85),int(59*1.15)),
    'randomforestclassifier__max_leaf_nodes':range(int(5500*.90),int(5500*1.10),int(5500/1000))
    
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    model, 
    param_distributions=param_distributions, 
    n_iter=330, 
    cv=10, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=22
)

In [18]:
%%time
search.fit(preped_data_train, y_train.to_numpy().reshape(-1,));

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Wall time: 3min 33s


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(memory='.\\memory\\',
                                      steps=[('randomforestclassifier',
                                              RandomForestClassifier())]),
                   n_iter=1, n_jobs=22,
                   param_distributions={'randomforestclassifier__max_depth': range(50, 67),
                                        'randomforestclassifier__max_leaf_nodes': range(4950, 6050, 5),
                                        'randomforestclassifier__min_samples_leaf': range(1, 3),
                                        'randomforestclassifier__min_samples_split': range(2, 4),
                                        'randomforestclassifier__n_estimators': [1000],
                                        'randomforestclassifier__n_jobs': [1]},
                   return_train_score=True, scoring='accuracy', verbose=10)

In [20]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', search.best_score_)

Best hyperparameters {'randomforestclassifier__n_jobs': 1, 'randomforestclassifier__n_estimators': 1000, 'randomforestclassifier__min_samples_split': 3, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__max_leaf_nodes': 5230, 'randomforestclassifier__max_depth': 64}
Cross-validation MAE 0.8172222222222223


In [21]:
test_wrangled= Base_Wrangle(test)
test_preped = prepipe.transform(test_wrangled)
submission_status_group=search.best_estimator_.predict(test_preped)

start: (14358, 29)
(14358, 29)
(14358, 30)
done


In [22]:
test_complete= test.copy(deep=True)
test_complete['status_group'] = submission_status_group
submission_df = test_complete.copy(deep=True)
submission_df = submission_df[['id','status_group']]
submission_df.set_index('id',inplace = True)
print(submission_df)
submission_df.to_csv('./submission.csv')

         status_group
id                   
50785      functional
51630      functional
17168      functional
45559  non functional
49871      functional
...               ...
39307  non functional
18990      functional
28749      functional
33492      functional
68707  non functional

[14358 rows x 1 columns]


In [23]:
new_best_est =search.best_estimator_
#print (new_best_est.get_params().keys())
new_best_est.set_params(randomforestclassifier__n_estimators=50000)
new_best_est.set_params(randomforestclassifier__n_jobs=20)
search2 = RandomizedSearchCV(new_best_est,n_iter=1, scoring='accuracy', cv=10,param_distributions={})
search2.fit(preped_data_train, y_train.to_numpy().reshape(-1,));

KeyboardInterrupt: 

In [ ]:
print('Best hyperparameters', search2.best_params_)
print('Cross-validation MAE', search2.best_score_)

In [ ]:
#model.fit(X_train, y_train,);

In [ ]:
def score_model(new_model):
    training_acc=new_model.score(X_train, y_train)
    validation_acc=new_model.score(X_val, y_val)

    print('Training Accuracy:', training_acc)
    print('Validation Accuracy:', validation_acc)

In [ ]:
score_model(model)